In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load dataset again
file_path = "/content/Housing.csv" # Corrected file path
df = pd.read_csv(file_path)

# Copy dataframe
data = df.copy()

# Encode categorical variables (yes/no -> 1/0)
binary_cols = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]
for col in binary_cols:
    data[col] = data[col].map({"yes": 1, "no": 0})

# One-hot encode furnishingstatus
data = pd.get_dummies(data, columns=["furnishingstatus"], drop_first=True)

# Features and target
X = data.drop("price", axis=1).values
y = data["price"].values.reshape(-1, 1)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add bias column (for intercept)
X = np.c_[np.ones(X.shape[0]), X]  # shape (n_samples, n_features+1)

# Parameters
m, n = X.shape
theta = np.zeros((n, 1))  # weights
alpha = 0.01
epochs = 1000

# Store cost values
cost_history = []

# Gradient Descent
for _ in range(epochs):
    y_pred = X @ theta
    error = y_pred - y
    grad = (1/m) * (X.T @ error)
    theta -= alpha * grad
    cost = np.mean(error**2) / 2
    cost_history.append(cost)

# Final weights
print("Trained parameters:", theta.ravel())

# Predictions
y_pred = X @ theta

# Mean Squared Error
mse = np.mean((y_pred - y)**2)
print("Mean Squared Error:", mse)

# Example prediction: predict price for first row
example = X[0].reshape(1, -1)
pred_price = example @ theta
print(f"Predicted price for first house: {pred_price[0][0]:.2f}")
print(f"Actual price: {y[0][0]}")

Trained parameters: [4766523.46205873  528392.45879117   86183.43839844  495886.32946498
  388929.25563021  147481.01859954  115723.73064647  165673.81594736
  179032.23841331  402616.22952824  238638.31973523  276487.14357135
  -22314.60382687 -192290.86300897]
Mean Squared Error: 1111192233533.0864
Predicted price for first house: 8133560.44
Actual price: 13300000


In [ ]:
# Get user input for prediction
print("Please enter the values for the following features:")

# Define the order of features based on the original dataframe columns (excluding 'price') and the order after one-hot encoding
feature_order = ['area', 'bedrooms', 'bathrooms', 'stories', 'mainroad', 'guestroom',
                 'basement', 'hotwaterheating', 'airconditioning', 'parking',
                 'prefarea', 'furnishingstatus_semi-furnished', 'furnishingstatus_unfurnished']

user_input = {}
for feature in feature_order:
    if feature in ['mainroad', 'guestroom', 'basement', 'hotwaterheating', 'airconditioning', 'prefarea']:
        while True:
            value = input(f"{feature} (yes/no): ").lower()
            if value in ["yes", "no"]:
                user_input[feature] = 1 if value == "yes" else 0
                break
            else:
                print("Invalid input. Please enter 'yes' or 'no'.")
    elif feature in ['furnishingstatus_semi-furnished', 'furnishingstatus_unfurnished']:
        # Handle furnishingstatus which was one-hot encoded. We need to know the original status.
        # A more robust way would be to ask for the original furnishingstatus and then encode it.
        # For simplicity here, we'll assume the user enters the one-hot encoded values directly.
        # A better approach for user input would be to ask for the original categorical value.
        # Let's ask for the original furnishing status and then encode it.
         pass # We will handle furnishing status separately
    else:
        while True:
            try:
                value = float(input(f"{feature}: "))
                user_input[feature] = value
                break
            except ValueError:
                print("Invalid input. Please enter a number.")

# Handle furnishingstatus input
while True:
    furnishing_status_input = input("furnishingstatus (furnished, semi-furnished, unfurnished): ").lower()
    if furnishing_status_input in ["furnished", "semi-furnished", "unfurnished"]:
        user_input['furnishingstatus_semi-furnished'] = 1 if furnishing_status_input == 'semi-furnished' else 0
        user_input['furnishingstatus_unfurnished'] = 1 if furnishing_status_input == 'unfurnished' else 0
        break
    else:
        print("Invalid input. Please enter 'furnished', 'semi-furnished', or 'unfurnished'.")

# Convert user input to a pandas DataFrame for preprocessing
user_df = pd.DataFrame([user_input])

# Ensure the columns are in the same order as the training data (excluding 'price')
# This is crucial for correct prediction
training_cols = data.drop("price", axis=1).columns
user_df = user_df.reindex(columns=training_cols, fill_value=0) # Fill any missing dummy columns with 0

# Preprocess the user input using the same scaler and adding the bias term
# We need to ensure the scaler was trained on the original feature set *before* adding the bias term
# Let's re-fit the scaler on the original features (without the bias term) from the training data
scaler_for_prediction = StandardScaler()
scaler_for_prediction.fit(data.drop("price", axis=1)) # Fit on the features used for training (before bias)

user_X = scaler_for_prediction.transform(user_df.values)
user_X = np.c_[np.ones(user_X.shape[0]), user_X] # Add bias term

# Make prediction
predicted_price = user_X @ theta

print(f"\nPredicted price: {predicted_price[0][0]:.2f}")

Please enter the values for the following features:


KeyboardInterrupt: Interrupted by user

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler

# Load dataset
file_path = "/content/Housing.csv" # Corrected file path
df = pd.read_csv(file_path)

# Copy dataframe
data = df.copy()

# Encode categorical variables (yes/no -> 1/0)
binary_cols = ["mainroad", "guestroom", "basement", "hotwaterheating", "airconditioning", "prefarea"]
for col in binary_cols:
    data[col] = data[col].map({"yes": 1, "no": 0})

# One-hot encode furnishingstatus
data = pd.get_dummies(data, columns=["furnishingstatus"], drop_first=True)

# Features and target
X = data.drop("price", axis=1).values
y = data["price"].values.reshape(-1, 1)

# Normalize features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# Add bias column (for intercept)
X = np.c_[np.ones(X.shape[0]), X]

# Parameters
m, n = X.shape
theta = np.zeros((n, 1))
alpha = 0.01
epochs = 1000

# Gradient Descent
for _ in range(epochs):
    y_pred = X @ theta
    error = y_pred - y
    grad = (1/m) * (X.T @ error)
    theta -= alpha * grad

print("Model trained successfully!")

# -------------------------------
# Prediction Function
# -------------------------------
def predict_price(area, bedrooms, bathrooms, stories,
                  mainroad, guestroom, basement, hotwaterheating,
                  airconditioning, parking, prefarea, furnishingstatus):
    """
    Predict house price given the features.
    furnishingstatus must be one of: 'furnished', 'semi-furnished', 'unfurnished'
    categorical yes/no inputs must be 1 (yes) or 0 (no)
    """

    # Build input vector (order must match training data)
    input_dict = {
        "area": area,
        "bedrooms": bedrooms,
        "bathrooms": bathrooms,
        "stories": stories,
        "mainroad": mainroad,
        "guestroom": guestroom,
        "basement": basement,
        "hotwaterheating": hotwaterheating,
        "airconditioning": airconditioning,
        "parking": parking,
        "prefarea": prefarea,
        "furnishingstatus_semi-furnished": 1 if furnishingstatus == "semi-furnished" else 0,
        "furnishingstatus_unfurnished": 1 if furnishingstatus == "unfurnished" else 0,
    }

    features = np.array(list(input_dict.values())).reshape(1, -1)

    # Scale features
    features_scaled = scaler.transform(features)

    # Add bias
    features_scaled = np.c_[np.ones(features_scaled.shape[0]), features_scaled]

    # Predict
    price = features_scaled @ theta
    return price[0][0]

# -------------------------------
# Example Usage
# -------------------------------
pred = predict_price(
    area=7500,
    bedrooms=4,
    bathrooms=2,
    stories=2,
    mainroad=1,
    guestroom=0,
    basement=1,
    hotwaterheating=0,
    airconditioning=1,
    parking=1,
    prefarea=1,
    furnishingstatus="semi-furnished"
)

print(f"Predicted Price: ₹{pred:.2f}")

Model trained successfully!
Predicted Price: ₹7729054.80
